In [88]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e1/sample_submission.csv
/kaggle/input/playground-series-s5e1/train.csv
/kaggle/input/playground-series-s5e1/test.csv


In [89]:
train =pd.read_csv("/kaggle/input/playground-series-s5e1/train.csv")

In [90]:
test =pd.read_csv("/kaggle/input/playground-series-s5e1/test.csv")

In [91]:
train.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0


In [92]:
!pip install scikit-learn

In [93]:
train.columns

Index(['id', 'date', 'country', 'store', 'product', 'num_sold'], dtype='object')

In [94]:
train.count()

id          230130
date        230130
country     230130
store       230130
product     230130
num_sold    221259
dtype: int64

In [95]:
train.isna().sum()

id             0
date           0
country        0
store          0
product        0
num_sold    8871
dtype: int64

In [96]:
train["country"].unique()

array(['Canada', 'Finland', 'Italy', 'Kenya', 'Norway', 'Singapore'],
      dtype=object)

In [97]:
for i in train.columns:
    print(i," : ",train[i].unique())

id  :  [     0      1      2 ... 230127 230128 230129]
date  :  ['2010-01-01' '2010-01-02' '2010-01-03' ... '2016-12-29' '2016-12-30'
 '2016-12-31']
country  :  ['Canada' 'Finland' 'Italy' 'Kenya' 'Norway' 'Singapore']
store  :  ['Discount Stickers' 'Stickers for Less' 'Premium Sticker Mart']
product  :  ['Holographic Goose' 'Kaggle' 'Kaggle Tiers' 'Kerneler'
 'Kerneler Dark Mode']
num_sold  :  [  nan  973.  906. ... 3446. 2266. 3996.]


In [98]:
unique_values = train["country"].unique()
country_dict = {value: index for index, value in enumerate(unique_values)}
country_dict

{'Canada': 0,
 'Finland': 1,
 'Italy': 2,
 'Kenya': 3,
 'Norway': 4,
 'Singapore': 5}

In [99]:
# df[col + '_encoded'] = df[col].map(label_dict)
train["country"] = train["country"].map(country_dict)

In [100]:
train["country"].unique()

array([0, 1, 2, 3, 4, 5])

In [101]:
train["store"].unique()

array(['Discount Stickers', 'Stickers for Less', 'Premium Sticker Mart'],
      dtype=object)

In [102]:
store_values = train["store"].unique()
store_dict = {value: index for index, value in enumerate(store_values)}
train["store"] = train["store"].map(store_dict)

In [103]:
train["store"].unique()

array([0, 1, 2])

In [104]:
product_values = train["product"].unique()
product_dict = {value: index for index, value in enumerate(product_values)}
train["product"] = train["product"].map(product_dict)

In [105]:
train["product"].unique()

array([0, 1, 2, 3, 4])

In [106]:
train.dropna(inplace=True)

In [107]:
train.drop("date",axis=1,inplace=True)

In [108]:
train.drop("id",axis=1,inplace=True)

In [109]:
train.isna().sum()

country     0
store       0
product     0
num_sold    0
dtype: int64

In [110]:
X = train.drop('num_sold', axis=1)  # Features
y = train['num_sold']  # Target variable

In [111]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [112]:
model_lr = LinearRegression()
model_lr.fit(X, y)

LinearRegression()

In [113]:
coefficients = model_lr.coef_
intercept = model_lr.intercept_

# Print the coefficients and the intercept
print("Coefficients (Weights):", coefficients)
print("Intercept:", intercept)

Coefficients (Weights): [ 54.4933336 273.986431    9.8135662]
Intercept: 314.71477294712247


In [114]:
# y_pred = model_lr.predict(X_val)

# # Evaluate the model
# mse = mean_squared_error(y_val, y_pred)
# r2 = r2_score(y_val, y_pred)

# print("Mean Squared Error:", mse)
# print("R^2 Score:", r2)

In [115]:
test["country"] = test["country"].map(country_dict)
test["store"] = test["store"].map(store_dict)
test["product"] = test["product"].map(product_dict)

In [116]:
ans=model_lr.predict(test.drop(["id","date"], axis=1))

In [117]:
submission = pd.DataFrame({"id": test.id, "num_sold": ans})
submission

,id,num_sold
0,230130,314.714773
1,230131,324.528339
2,230132,334.341905
3,230133,344.155472
4,230134,353.969038
...,...,...
98545,328675,1135.154303
98546,328676,1144.967869
98547,328677,1154.781435
98548,328678,1164.595002


In [118]:
submission.to_csv('submission.csv', index = False)

In [119]:
from sklearn.ensemble import RandomForestRegressor

model_rf=RandomForestRegressor(n_estimators=100, random_state=42)

In [121]:
model_rf.fit(X, y)

RandomForestRegressor(random_state=42)

In [123]:
ans_rf=model_rf.predict(test.drop(["id","date"], axis=1))

In [124]:
submission_rf = pd.DataFrame({"id": test.id, "num_sold": ans_rf})
submission_rf.to_csv('submission_rf.csv', index = False)

### Converting date into minutes

In [125]:
train =pd.read_csv("/kaggle/input/playground-series-s5e1/train.csv")

In [126]:
test =pd.read_csv("/kaggle/input/playground-series-s5e1/test.csv")

In [127]:
train["store"] = train["store"].map(store_dict)
train["country"] = train["country"].map(country_dict)
train["product"] = train["product"].map(product_dict)

In [128]:
test["country"] = test["country"].map(country_dict)
test["store"] = test["store"].map(store_dict)
test["product"] = test["product"].map(product_dict)

In [129]:
train.dropna(inplace=True)

In [130]:
train["date"].unique()

array(['2010-01-01', '2010-01-02', '2010-01-03', ..., '2016-12-29',
       '2016-12-30', '2016-12-31'], dtype=object)

In [131]:
train.columns

Index(['id', 'date', 'country', 'store', 'product', 'num_sold'], dtype='object')

In [132]:

train['date'] = pd.to_datetime(train['date'])

# Choose a reference date (e.g., the first date in the column)
reference_date = train['date'].min()

# Calculate the difference in days
train['date'] = (train['date'] - reference_date).dt.days

In [133]:

# Convert the date column to datetime
test['date'] = pd.to_datetime(test['date'])


# Calculate the difference in days
test['date'] = (test['date'] - reference_date).dt.days

In [134]:
test.columns

Index(['id', 'date', 'country', 'store', 'product'], dtype='object')

In [141]:
from sklearn.ensemble import RandomForestRegressor

model_rf_days=RandomForestRegressor(n_estimators=400, random_state=62)
model_rf_days.fit(train.drop(["id","num_sold"], axis=1), train["num_sold"])

RandomForestRegressor(n_estimators=400, random_state=62)

In [142]:
ans_rf_days=model_rf_days.predict(test.drop("id", axis=1))

In [143]:
submission_rf_days = pd.DataFrame({"id": test.id, "num_sold": ans_rf_days})
submission_rf_days.to_csv('submission_rf_days.csv', index = False)